### Imports

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

### Import data

In [3]:
df_train = pd.read_csv('data/data_train.csv')
df_train.head()

df_test = pd.read_csv('data/data_validation.csv')
df_test.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
2,769,1,2.9,1,0,0,9,0.1,182,5,...,248,874,3946,5,2,7,0,0,0,3
3,775,0,1.0,0,3,0,46,0.7,159,2,...,862,1864,568,17,15,11,1,1,1,0
4,595,0,0.9,1,7,1,23,0.1,121,3,...,441,810,3752,10,2,18,1,1,0,3


In [4]:
x_train = df_train.drop('price_range',axis=1)
y_train = df_train['price_range']

x_test = df_test.drop('price_range',axis=1)
y_test = df_test['price_range']

x_train

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,804,1,0.8,1,12,1,41,0.9,89,1,13,709,818,2027,11,5,11,1,0,0
1,1042,0,2.2,0,15,1,11,0.6,139,5,16,68,1018,2826,18,0,2,1,0,0
2,1481,1,2.0,1,0,0,35,0.5,105,3,0,249,522,2635,17,16,4,1,0,1
3,1104,0,1.7,0,1,1,60,0.4,199,2,13,653,1413,1229,6,0,3,1,1,1
4,652,0,0.5,1,1,0,58,0.6,142,3,2,464,781,565,18,12,9,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,536,1,1.4,0,0,1,53,0.7,135,3,0,547,705,1211,15,10,7,1,0,1
1396,1097,0,0.8,0,10,1,21,0.1,160,7,15,1277,1352,2219,15,6,12,1,0,1
1397,1179,1,0.5,0,7,1,32,0.3,182,2,12,85,1451,340,16,5,16,1,0,0
1398,719,1,0.5,1,0,1,23,0.4,113,6,9,431,1727,3990,14,9,12,1,1,1


### Find numeric and categorical columns

In [5]:
numeric_columns = []
categorical_columns = []

for column in x_train:
  column_values = df_train[column]
  count_unique_values = len(np.unique(column_values))

  if count_unique_values < 0.05 * len(column_values):
    categorical_columns.append(column)
  else:
    numeric_columns.append(column)

print(f"{len(categorical_columns)} Categorical columns =", categorical_columns)
print(f"{len(numeric_columns)} Numeric columns =", numeric_columns)

15 Categorical columns = ['blue', 'clock_speed', 'dual_sim', 'fc', 'four_g', 'int_memory', 'm_dep', 'n_cores', 'pc', 'sc_h', 'sc_w', 'talk_time', 'three_g', 'touch_screen', 'wifi']
5 Numeric columns = ['battery_power', 'mobile_wt', 'px_height', 'px_width', 'ram']


In [6]:
num_columns = ["battery_power", "clock_speed", "fc", "int_memory", "m_dep", "mobile_wt", "n_cores", "pc", "px_height", "px_width", "ram", "sc_h", "sc_w", "talk_time", "price_range"]
non_num_columns = ["blue", "dual_sim", "four_g", "three_g", "touch_screen", "wifi"]

numeric_columns = num_columns
categorical_columns = non_num_columns

In [7]:
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from lib.naive_bayes import NaiveBayesClassifier, NaiveBayesCategorical

# Numeric columns
model_gnb = GaussianNB()
model_gnb.fit(df_train[numeric_columns], y_train)

# Categorical columns
model_cnb = CategoricalNB()
model_cnb.fit(df_train[categorical_columns], y_train)

manual_nb = NaiveBayesCategorical()
manual_nb.fit(df_train[categorical_columns].values.T, y_train.values)
category_manual = manual_nb.predict_proba(df_test[categorical_columns].values)
print(category_manual)

categorical_posteriors = model_cnb.predict_proba(df_test[categorical_columns])
print(categorical_posteriors)
numerical_posteriors   = model_gnb.predict_proba(df_test[numeric_columns])

combined_probabilities = categorical_posteriors * numerical_posteriors

# Get the class with the highest combined probability as the final prediction
final_predictions = np.argmax(combined_probabilities, axis=1)
final_predictions

[[1 0 1 ... 1 1 0]
 [1 0 1 ... 0 1 0]
 [1 1 0 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [0 0 0 ... 0 1 0]
 [0 0 1 ... 0 1 1]]
{0: 0.254985754985755, 1: 0.2535612535612536, 2: 0.24572649572649571, 3: 0.24287749287749288}
[{'0-0': 0.525, '0-1': 0.5279329608938548, '0-2': 0.49855907780979825, '0-3': 0.47230320699708456, '1-0': 0.475, '1-1': 0.4720670391061452, '1-2': 0.5014409221902018, '1-3': 0.5276967930029155}, {'0-0': 0.4777777777777778, '0-1': 0.4972067039106145, '0-2': 0.5331412103746398, '0-3': 0.48104956268221577, '1-0': 0.5222222222222223, '1-1': 0.5027932960893855, '1-2': 0.4668587896253602, '1-3': 0.5189504373177842}, {'0-0': 0.4666666666666667, '0-1': 0.46089385474860334, '0-2': 0.5014409221902018, '0-3': 0.4518950437317784, '1-0': 0.5333333333333333, '1-1': 0.5391061452513967, '1-2': 0.49855907780979825, '1-3': 0.5481049562682215}, {'0-0': 0.2777777777777778, '0-1': 0.2122905027932961, '0-2': 0.23631123919308358, '0-3': 0.23615160349854228, '1-0': 0.7222222222222222, '1-1': 0.7877094972

array([1, 2, 3, 0, 3, 1, 3, 0, 3, 2, 3, 2, 3, 0, 3, 0, 2, 1, 1, 2, 3, 2,
       0, 1, 2, 0, 3, 1, 0, 3, 1, 3, 3, 0, 2, 3, 1, 3, 2, 1, 1, 2, 0, 2,
       3, 1, 1, 2, 2, 3, 2, 2, 3, 0, 1, 3, 1, 3, 3, 2, 2, 3, 3, 1, 3, 2,
       3, 2, 3, 3, 2, 3, 1, 0, 1, 2, 0, 3, 1, 0, 3, 3, 1, 2, 3, 2, 3, 3,
       0, 2, 1, 1, 1, 2, 2, 1, 3, 2, 0, 3, 3, 3, 1, 2, 3, 1, 3, 3, 3, 3,
       3, 3, 2, 2, 3, 1, 0, 2, 1, 3, 1, 2, 2, 3, 3, 0, 2, 2, 1, 3, 3, 1,
       0, 0, 3, 1, 0, 2, 3, 0, 1, 3, 3, 1, 2, 3, 1, 2, 1, 1, 3, 0, 0, 2,
       1, 1, 2, 0, 1, 3, 3, 3, 0, 2, 3, 0, 0, 2, 1, 2, 2, 1, 0, 1, 0, 1,
       3, 1, 2, 0, 3, 1, 1, 2, 0, 2, 0, 3, 2, 0, 3, 2, 0, 0, 2, 0, 1, 3,
       3, 1, 1, 2, 2, 3, 2, 3, 3, 3, 0, 2, 1, 2, 3, 1, 1, 2, 3, 0, 0, 2,
       2, 1, 2, 3, 1, 2, 0, 3, 0, 2, 2, 2, 2, 2, 3, 0, 2, 3, 3, 3, 0, 0,
       1, 0, 3, 3, 1, 1, 0, 2, 2, 1, 3, 3, 0, 2, 2, 0, 0, 1, 0, 2, 2, 3,
       3, 0, 2, 3, 3, 0, 0, 2, 1, 2, 1, 0, 3, 2, 2, 2, 0, 1, 0, 2, 1, 1,
       0, 1, 3, 1, 3, 0, 1, 0, 1, 3, 2, 3, 0, 3, 1,

In [8]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

def evaluate_classifier_performance(prediction, y_test):
    print('Accuracy Average:', accuracy_score(y_test, prediction))
    print('F1 Macro Average:', f1_score(y_test, prediction, average='macro'))
    print('F1 Micro Average:', f1_score(y_test, prediction, average='micro'))
    print('Precision Macro Average:', precision_score(y_test, prediction, average='macro',zero_division=0))
    print('Precision Micro Average:', precision_score(y_test, prediction, average='micro',zero_division=0))
    print('Recall Macro Average:', recall_score(y_test, prediction, average='macro',zero_division=0))
    print('Recall Micro Average:', recall_score(y_test, prediction, average='micro',zero_division=0))

In [9]:
evaluate_classifier_performance(final_predictions,y_test)

Accuracy Average: 1.0
F1 Macro Average: 1.0
F1 Micro Average: 1.0
Precision Macro Average: 1.0
Precision Micro Average: 1.0
Recall Macro Average: 1.0
Recall Micro Average: 1.0
